In [1]:
import pandas as pd

In [2]:
import time

In [3]:
import get_kepler

In [4]:
import numpy as np
from astropy.timeseries import LombScargle
from astropy import units as u
import matplotlib.pyplot as plt

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
KIC       105 non-null int64
Period    105 non-null float64
teff      105 non-null int64
logg      105 non-null float64
radius    105 non-null float64
mass      105 non-null float64
dens      105 non-null float64
dtypes: float64(5), int64(2)
memory usage: 5.8 KB


In [11]:
data = get_kepler.read_csv(5697805)

7220320
Optimal parameters not found: Number of calls to function has reached maxfev = 7000.
0
4995401
10753751
8081482
11086270
3662635
6041734


In [12]:
data["period"]

,KIC,Period
0,9973591,15.116
0,10054132,15.798
0,6877490,11.546
0,8782968,23.386
0,10653094,12.016


In [7]:
data["periods"]

[0.010210159726767505,
 0.010223750233830188,
 0.020434891953662427,
 0.010209625752077445,
 0.010208229051540754,
 0.010223602392182753,
 0.010223781532634615,
 0.010209699379121714,
 0.010224067233680165,
 0.010219627603662226,
 0.02043493632791364,
 0.010211784645562026,
 0.010222693735549222,
 0.010217250530153656,
 0.010221289537361384]

In [1]:
from pwkit import pdm